# Plot global overturning circulation

In [1]:
import cosima_cookbook as cc
from dask.distributed import Client

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cmocean as cm
import matplotlib.gridspec as gridspec
%matplotlib inline

# Stop annoying warnings coming out of xarray.
import warnings
warnings.filterwarnings('ignore')

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 128.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35965,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 128.00 GiB
Comm: tcp://127.0.0.1:43665,Total threads: 4
Dashboard: /proxy/42473/status,Memory: 18.29 GiB
Nanny: tcp://127.0.0.1:39133,


In [3]:
# CM2 database
session_CM2 = cc.database.create_session('/g/data/p73/archive/non-CMIP/ACCESS-CM2/CM2.db')
expt_CM2_025 = 'cj877'
expt_CM2_1   = 'bz687'

In [4]:
# OM2 database
session_OM2  = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')
expt_OM2_025 = '025deg_jra55_ryf9091_gadi'
expt_OM2_1   = '1deg_jra55_ryf9091_gadi'

In [5]:
path_to_data = '/g/data/x77/wgh581/Post_Process/ACCESS_CM2_025/'

If you get a 'read-only database' warning, it is because the database has not finished building. 

In [6]:
# Plotting info
ft_size = 16
plt.rcParams.update({'font.size': ft_size})
fig_path = '/g/data/x77/wgh581/Figures/Figures_ACCESS_CM2/'

### Calculate overturning

In [7]:
def compute_psi_rho(expt, session, frequency='1 monthly', nbound=None, start_time=None, end_time=None):
    rho = 1025 # mean density of sea-water in kg/m^3
    
    varlist = cc.querying.get_variables(session, expt)
    if varlist['name'].str.contains('ty_trans_rho_gm').any():
        GM = True
        print('GM is True')
        psiGM = cc.querying.getvar(expt, 'ty_trans_rho_gm', session, frequency=frequency, n=nbound, start_time=start_time, end_time=end_time)
        psiGM = psiGM.sum('grid_xt_ocean')
        psiGM = psiGM / (1e6*rho)
    else:
        GM = False
        print('GM is False')
        
    psi = cc.querying.getvar(expt, 'ty_trans_rho', session, frequency=frequency, n=nbound, start_time=start_time, end_time=end_time)      
    psi = psi / (1e6*rho) # converts kg/s to Sv
    psi = psi.sum('grid_xt_ocean').cumsum('potrho').mean(dim='time').load() 
    if GM:
        psi = psi + psiGM.mean('time')
        
    return psi.compute()

In [ ]:
year = np.arange(1, 500, 1)

for ii in year:
    
    if ii > 341:
        if len(str(int(ii))) == 1:
            year_tmp = '000' +str(ii)
        elif len(str(int(ii))) == 2:
            year_tmp = '00' + str(ii)
        elif len(str(int(ii))) == 3:
            year_tmp = '0' + str(ii)

        start_time = year_tmp + '-01-01'
        end_time   = year_tmp + '-12-31'
        data_name   = 'Overturning_' + year_tmp + '.nc'

        psi_tmp = compute_psi_rho(expt_CM2_025, session_CM2, start_time=start_time, end_time=end_time)
        psi_tmp.to_netcdf(path_to_data + 'overturning_tmp/' + data_name)
        print('Finished year ', year_tmp)

GM is True


/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/xarray/coding/times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/xarray/coding/times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/xarray/coding/times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datet

Finished year  0342
GM is True


/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/xarray/coding/times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/xarray/core/indexing.py:423: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/xarray/coding/times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, cal

Finished year  0343
GM is True
Finished year  0344
GM is True
Finished year  0345
GM is True
Finished year  0346
GM is True
Finished year  0347
GM is True
Finished year  0348
GM is True
Finished year  0349
GM is True
Finished year  0350
GM is True
Finished year  0351
GM is True
Finished year  0352
GM is True
Finished year  0353
GM is True
Finished year  0354
GM is True
Finished year  0355
GM is True


...

Finished year  0399
GM is True


2022-11-24 14:29:10,383 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2022-11-24 14:29:11,374 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2022-11-24 14:29:24,218 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


Finished year  0400
GM is True


2022-11-24 14:30:00,674 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2022-11-24 14:30:12,668 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


Finished year  0401
GM is True
Finished year  0402
GM is True
Finished year  0403
GM is True
Finished year  0404
GM is True
Finished year  0405
GM is True
Finished year  0406
GM is True
Finished year  0407
GM is True
Finished year  0408
GM is True
Finished year  0409
GM is True
Finished year  0410
GM is True
Finished year  0411
GM is True
Finished year  0412
GM is True
Finished year  0413
GM is True
Finished year  0414
GM is True
Finished year  0415
GM is True
Finished year  0416
GM is True
Finished year  0417
GM is True
Finished year  0418
GM is True
Finished year  0419
GM is True
Finished year  0420
GM is True
Finished year  0421
GM is True
Finished year  0422
GM is True
Finished year  0423
GM is True
Finished year  0424
GM is True
Finished year  0425
GM is True
Finished year  0426
GM is True
Finished year  0427
GM is True
Finished year  0428
GM is True
Finished year  0429
GM is True
Finished year  0430
GM is True
Finished year  0431
GM is True
Finished year  0432
GM is True
Finished